[View in Colaboratory](https://colab.research.google.com/github/animuku/CIFAR-10-Using-ResNet/blob/master/ImageRecognition.ipynb)

In [3]:
!wget -c https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

--2018-08-20 14:52:37--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  2.03MB/s    in 54s     

2018-08-20 14:53:31 (3.04 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]



In [4]:
!tar xvzf cifar-10-python.tar.gz


cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [55]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
K.set_image_dim_ordering('th')
print(keras.__version__)




2.1.6


In [0]:
img_size = 32
num_channels=3
img_size_flat = img_size * img_size * num_channels
num_classes = 10
num_files_train = 5
num_images_file=10000
num_images_train=num_files_train*num_images_file


In [0]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [0]:
def convert_images(raw):
    raw_float=np.array(raw,dtype=float)/255.0
    images=raw_float.reshape([-1,num_channels,img_size,img_size])
    images = images.transpose([0, 2, 3, 1])
    return images


In [0]:
def load_data(filename):
    data=unpickle(filename)
    raw_images=data[b'data']
    cls = np.array(data[b'labels'])
    
    images=convert_images(raw_images)
    return images,cls

In [0]:
def load_training_data():
    images=np.zeros(shape=[num_images_train,img_size,img_size,num_channels],dtype=float)
    classes=np.zeros(shape=[num_images_train],dtype=int)
    
    begin=0
    
    for i in range(num_files_train):
        images_batch, cls_batch = load_data(filename='cifar-10-batches-py/data_batch_' + str(i + 1))
        
        num_images=len(images_batch)
        end=begin+num_images
        
        images[begin:end,:]=images_batch
        classes[begin:end]=cls_batch
        
        begin=end
        
        return images, classes

In [0]:
def load_test_data():
    images, cls = load_data(filename="cifar-10-batches-py/test_batch")
    return images,cls

In [83]:
images,classes=load_training_data()
classes=to_categorical(classes,10)
print(images.shape)

test_images,test_classes=load_test_data()
test_classes=to_categorical(test_classes,10)
print(classes.shape)


(50000, 32, 32, 3)
(50000, 10)


In [14]:
print (images[2,:,:,:])
print(test_images[2,:,:,:])

[[[1.         1.         1.        ]
  [0.99215686 0.99215686 0.99215686]
  [0.99215686 0.99215686 0.99215686]
  ...
  [0.99215686 0.99215686 0.99215686]
  [0.99215686 0.99215686 0.99215686]
  [0.99215686 0.99215686 0.99215686]]

 [[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]]

 [[1.         1.         1.        ]
  [0.99607843 0.99607843 0.99607843]
  [0.99607843 0.99607843 0.99607843]
  ...
  [0.99607843 0.99607843 0.99607843]
  [0.99607843 0.99607843 0.99607843]
  [0.99607843 0.99607843 0.99607843]]

 ...

 [[0.44313725 0.47058824 0.43921569]
  [0.43529412 0.4627451  0.43529412]
  [0.41176471 0.43921569 0.41568627]
  ...
  [0.28235294 0.31764706 0.31372549]
  [0.28235294 0.31372549 0.30980392]
  [0.28235294 0.31372549 0.30980392]]

 [[0.43529412 0.4627451  0.43137255]
  [0.40784314 0.43529412 0.40784314]


In [0]:
def identity_block(X,f,filters):
    F1,F2,F3=filters
    
    X_shortcut=X
    
    X=Conv2D(filters=F1,kernel_size=(1,1),strides=(1,1),padding='valid')(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same')(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid')(X)
    X=BatchNormalization(axis=3)(X)
    
    
    X=Add()([X,X_shortcut])
    
    X=Activation('relu')(X)
    
    return X   

In [0]:
def convolutional_block(X,f,filters,s=2):
    F1,F2,F3=filters
    
    X_shortcut=X
    
    X=Conv2D(filters=F1,kernel_size=(1,1),strides=(s,s))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same')(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid')(X)
    X=BatchNormalization(axis=3)(X)
    
    
    X_shortcut=Conv2D(filters=F3,kernel_size=(1,1),strides=(s,s),padding='valid')(X_shortcut)
    X_shortut=BatchNormalization(axis=3)(X_shortcut)
    
    X=Add()([X,X_shortcut])
    
    X=Activation('relu')(X)
    
    return X   
  
  

In [0]:
def ResNet50(input_shape = (32, 32, 3), classes = 10):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2),padding='same',name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2),dim_ordering="th")(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256],s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X,f=3,filters=[128,128,512],s=2)
    X = identity_block(X,3,[128,128,512])
    X = identity_block(X,3,[128,128,512])
    X = identity_block(X,3,[128,128,512])

    # Stage 4 (≈6 lines)
    X = convolutional_block(X,f=3,filters=[256,256,1024],s=2)
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])

    # Stage 5 (≈3 lines)
    X = convolutional_block(X,f=3,filters=[512,512,2048],s=2)
    X = identity_block(X,3,[512,512,2048])
    X = identity_block(X,3,[512,512,2048])

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2,2),name='avg_pool',padding='same')(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model
  
  

In [93]:
model=ResNet50(input_shape=(32,32,3),classes=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((3, 3), strides=(2, 2), data_format="channels_first")`


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [96]:
model.fit(images,classes,epochs = 5,batch_size = 32)

Epoch 1/5
50000/50000 [==============================] - 499s 10ms/step - loss: 2.8996 - acc: 0.8201
Epoch 2/5
15584/50000 [========>.....................] - ETA: 5:43 - loss: 2.9084 - acc: 0.8196

KeyboardInterrupt: ignored

In [72]:
preds = model.evaluate(test_images,test_classes)

10000/10000 [==============================] - 14s 1ms/step


In [73]:
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

Loss = 14.506285661315918
Test Accuracy = 0.1
